参考：
1. https://blog.csdn.net/qq_40210472/article/details/88995433
2. https://www.jianshu.com/p/c2a9d3e76706

glove和word2vec是目前最常用的两个训练词向量的模型，两者训练出来的文件都以文本格式呈现，区别在于word2vec包含向量的数量及其维度。
下面举个例子。

glove词向量的格式如下：
```
word1 0.123 0.134 0.532 0.152
word2 0.934 0.412 0.532 0.159
word3 0.334 0.241 0.324 0.18
...
word9 0.334 0.241 0.324 0.188
```
word2vec词向量的格式：
```
9 4   # 这一行包含向量的数量及其维度
word1 0.123 0.134 0.532 0.152
word2 0.934 0.412 0.532 0.159
word3 0.334 0.241 0.324 0.188
...
word9 0.334 0.241 0.324 0.188
```


# gensim加载glove训练的词向量
由于glove模型目前只有一个C的版本，而且也未添加到其他库中。

gensim库添加了一个模块，可以用来将glove格式的词向量转为word2vec的词向量，具体操作如下：

In [2]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors


In [5]:
# 输入文件
glove_file = datapath('D:\BaiduNetdiskDownload\glove.6B\glove.6B.50d.txt')
# 输出文件
tmp_file = get_tmpfile("D:\BaiduNetdiskDownload\glove.6B\\test_word2vec.txt")


# 开始转换
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

# 加载转化后的文件
model = KeyedVectors.load_word2vec_format(tmp_file)

In [7]:
model['you']

array([-1.0919e-03,  3.3324e-01,  3.5743e-01, -5.4041e-01,  8.2032e-01,
       -4.9391e-01, -3.2588e-01,  1.9972e-03, -2.3829e-01,  3.5554e-01,
       -6.0655e-01,  9.8932e-01, -2.1786e-01,  1.1236e-01,  1.1494e+00,
        7.3284e-01,  5.1182e-01,  2.9287e-01,  2.8388e-01, -1.3590e+00,
       -3.7951e-01,  5.0943e-01,  7.0710e-01,  6.2941e-01,  1.0534e+00,
       -2.1756e+00, -1.3204e+00,  4.0001e-01,  1.5741e+00, -1.6600e+00,
        3.7721e+00,  8.6949e-01, -8.0439e-01,  1.8390e-01, -3.4332e-01,
        1.0714e-02,  2.3969e-01,  6.6748e-02,  7.0117e-01, -7.3702e-01,
        2.0877e-01,  1.1564e-01, -1.5190e-01,  8.5908e-01,  2.2620e-01,
        1.6519e-01,  3.6309e-01, -4.5697e-01, -4.8969e-02,  1.1316e+00],
      dtype=float32)

至此，我们就可以完美的用gensim加载glove训练的词向量了。



# 转化为pytorch的Embedding

In [8]:
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir='./logs')
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
import os
import json


In [11]:
import gensim.models.word2vec as w2v


In [20]:
for i in model.vocab:
    print(i)
    break
vocab_size = len(model.vocab) + 1
embed_size = 50
weight = torch.zeros(vocab_size, embed_size)
    




the


In [21]:
vocab = list(model.vocab)

In [22]:
len(vocab)

400000

In [23]:
word2id = dict()

In [24]:
for index,word in enumerate(vocab):
    word2id[word]=index
    weight[index] = torch.from_numpy(model[word])

In [27]:
weight[2]

tensor([ 1.5164e-01,  3.0177e-01, -1.6763e-01,  1.7684e-01,  3.1719e-01,
         3.3973e-01, -4.3478e-01, -3.1086e-01, -4.4999e-01, -2.9486e-01,
         1.6608e-01,  1.1963e-01, -4.1328e-01, -4.2353e-01,  5.9868e-01,
         2.8825e-01, -1.1547e-01, -4.1848e-02, -6.7989e-01, -2.5063e-01,
         1.8472e-01,  8.6876e-02,  4.6582e-01,  1.5035e-02,  4.3474e-02,
        -1.4671e+00, -3.0384e-01, -2.3441e-02,  3.0589e-01, -2.1785e-01,
         3.7460e+00,  4.2284e-03, -1.8436e-01, -4.6209e-01,  9.8329e-02,
        -1.1907e-01,  2.3919e-01,  1.1610e-01,  4.1705e-01,  5.6763e-02,
        -6.3681e-05,  6.8987e-02,  8.7939e-02, -1.0285e-01, -1.3931e-01,
         2.2314e-01, -8.0803e-02, -3.5652e-01,  1.6413e-02,  1.0216e-01])

In [30]:
vocab[2]

'.'

In [31]:
weight[word2id[vocab[2]]] == weight[2]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1], dtype=torch.uint8)

In [32]:
embedding = nn.Embedding.from_pretrained(weight)

In [33]:
embedding(torch.LongTensor([1,2,3,4])).size()

torch.Size([4, 50])